# Even reduction

In [1]:
using DynamicPolynomials
@polyvar x

(x,)

We would like to find the minimum value of the following polynomial:

In [2]:
poly = x^4 - 2x^2

using SumOfSquares

We define the custom action as follows:

In [3]:
struct OnSign <: Symmetry.OnMonomials end
using PermutationGroups
import SymbolicWedderburn
SymbolicWedderburn.coeff_type(::OnSign) = Float64
function SymbolicWedderburn.action(::OnSign, p::Permutation, mono::AbstractMonomial)
    if isone(p) || iseven(DynamicPolynomials.degree(mono))
        return 1 * mono
    else
        @assert p.perm == perm"(1,2)"
        return -1 * mono
    end
end
G = PermGroup([perm"(1,2)"])

Permutation group on 1 generator generated by
 (1,2)

We can exploit the symmetry as follows:

In [4]:
import Clarabel
solver = Clarabel.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, OnSign())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 5
  constraints   = 9
  nnz(P)        = 0
  nnz(A)        = 11
  cones (total) = 3
    : Zero        = 1,  numel = 5
    : Nonnegative = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter  

-0.9999999977220708

We indeed find `-1`, let's verify that symmetry was exploited:

In [5]:
gram_matrix(con_ref)

BlockDiagonalGramMatrix with 2 blocks:
[1] Block with row/column basis:
     Simple basis:
  FixedBasis([1.0·1 + 0.0·x + 0.0·x², 0.0·1 + 0.0·x + 1.0·x²])
    And entries in a 2×2 SymMatrix{Float64}:
      0.9999999977220708  -1.000000000898501
     -1.000000000898501    1.0
[2] Block with row/column basis:
     Simple basis:
  FixedBasis([0.0·1 + 1.0·x + 0.0·x²])
    And entries in a 1×1 SymMatrix{Float64}:
     1.7970022023157515e-9

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*